<a href="https://colab.research.google.com/github/Glory-AI/Glory-AI/blob/main/Glory_Akanbi_HAckathon_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer, mean_squared_error

In [20]:
train= pd.read_csv('/content/train.csv')
test= pd.read_csv("/content/test.csv")

In [21]:
test_ids = test["id"]

In [22]:
for col in ["fuel_type", "clean_title", "engine", "transmission", "ext_col", "int_col", "accident"]:
    mode_val = train[col].mode()[0]
    train[col].fillna(mode_val, inplace=True)
    test[col].fillna(mode_val, inplace=True)

train["accident"].fillna("None reported", inplace=True)
test["accident"].fillna("None reported", inplace=True)

train['clean_title_flag'] = train['clean_title'].apply(lambda x: 1 if x == "Yes" else 0)
test['clean_title_flag'] = test['clean_title'].apply(lambda x: 1 if x == "Yes" else 0)
# Age of car = current_year - model_year
CURRENT_YEAR = 2025
train["car_age"] = 2025 - train["model_year"]
test["car_age"] = 2025- test["model_year"]

# Drop unused columns
y = train["price"]
X = train.drop(columns=["price", "id", "clean_title", "model_year"])
X_test= test.drop(columns=["id", "clean_title", "model_year"])

/tmp/ipython-input-4240312746.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train[col].fillna(mode_val, inplace=True)
/tmp/ipython-input-4240312746.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                188533 non-null  int64 
 1   brand             188533 non-null  object
 2   model             188533 non-null  object
 3   model_year        188533 non-null  int64 
 4   milage            188533 non-null  int64 
 5   fuel_type         188533 non-null  object
 6   engine            188533 non-null  object
 7   transmission      188533 non-null  object
 8   ext_col           188533 non-null  object
 9   int_col           188533 non-null  object
 10  accident          188533 non-null  object
 11  clean_title       188533 non-null  object
 12  price             188533 non-null  int64 
 13  clean_title_flag  188533 non-null  int64 
 14  car_age           188533 non-null  int64 
dtypes: int64(6), object(9)
memory usage: 21.6+ MB


In [24]:
print(train.isnull().sum())

id                  0
brand               0
model               0
model_year          0
milage              0
fuel_type           0
engine              0
transmission        0
ext_col             0
int_col             0
accident            0
clean_title         0
price               0
clean_title_flag    0
car_age             0
dtype: int64


In [25]:
categorical_cols = ["brand", "model", "fuel_type", "transmission",
                    "ext_col", "int_col", "engine", "accident"]
numeric_cols = [col for col in X.columns if col not in categorical_cols]

# Transformer: One-hot encode categoricals, pass through numeric
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

In [26]:
rf = RandomForestRegressor(
    n_estimators=100,     # keep trees reasonable (try 50 if still slow)
    max_depth=15,         # limit depth
    max_features="sqrt",  # fewer features per split = faster
    n_jobs=-1,
    random_state=42
)

model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", rf)
])

# Cross-validation
cv_scores = cross_val_score(
    model, X, y, cv=3,   # reduce folds to 3 if you just need results today
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

print("Baseline CV RMSE scores:", -cv_scores)
print("Baseline Average RMSE:", -cv_scores.mean())

Baseline CV RMSE scores: [76132.39417687 76259.80028303 73890.82574954]
Baseline Average RMSE: 75427.67340314454


In [28]:
model.fit(X, y)

# === Predict on test set ===
test_preds = model.predict(X_test)

# === Prepare submission ===
submission = pd.DataFrame({
    "id": test["id"],
    "price": test_preds
})

# Save to CSV
submission.to_csv("submission.csv", index=False)


In [29]:
print(submission.shape)

(125690, 2)
